<a href="https://colab.research.google.com/github/KSR313/Ml_TimeSeries/blob/main/ML_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://builtin.com/data-science/time-series-forecasting-python)

# **A Guide to Time Series Forecasting in Python:**

In [2]:
import pandas_datareader.data as web
import datetime

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
btc = web.get_data_yahoo(['BTC-USD'])
#  , start=datetime.datetime(2018, 1, 1), end=datetime.datetime(2020, 12, 2))['Close']
print(btc.head())

TypeError: ignored

In [ ]:
btc.to_csv("btc.csv")

In [ ]:
btc = pd.read_csv("btc.csv")
print(btc.head())

In [ ]:
btc.index = pd.to_datetime(btc['Date'], format='%Y-%m-%d')

In [ ]:
del btc['Date']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Splitting Data

In [ ]:
plt.ylabel('BTC Price')
plt.xlabel('Date')
plt.xticks(rotation=45)

In [ ]:
plt.plot(btc.index, btc['BTC-USD'], )

In [ ]:
train = btc[btc.index < pd.to_datetime("2020-11-01", format='%Y-%m-%d')]
test = btc[btc.index > pd.to_datetime("2020-11-01", format='%Y-%m-%d')]

plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.ylabel('BTC Price')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Train/Test split for BTC Data")
plt.show()

Autoregressive Moving Average (ARMA)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [1]:
y = train['BTC-USD']

In [ ]:
ARMAmodel = SARIMAX(y, order = (1, 0, 1))

In [1]:
ARMAmodel = ARMAmodel.fit()

In [ ]:
y_pred = ARMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05)
y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"]

In [ ]:
plt.plot(y_pred_out, color='green', label = 'Predictions')
plt.legend()

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

arma_rmse = np.sqrt(mean_squared_error(test["BTC-USD"].values, y_pred_df["Predictions"]))
print("RMSE: ",arma_rmse)

Autoregressive Integrated Moving Average (ARIMA)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [1]:
ARIMAmodel = ARIMA(y, order = (2, 2, 2))
ARIMAmodel = ARIMAmodel.fit()

y_pred = ARIMAmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05)
y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"]
plt.plot(y_pred_out, color='Yellow', label = 'ARIMA Predictions')
plt.legend()


import numpy as np
from sklearn.metrics import mean_squared_error

arma_rmse = np.sqrt(mean_squared_error(test["BTC-USD"].values, y_pred_df["Predictions"]))
print("RMSE: ",arma_rmse)

Seasonal ARIMA (SARIMA)

In [ ]:
SARIMAXmodel = SARIMAX(y, order = (5, 4, 2), seasonal_order=(2,2,2,12))
SARIMAXmodel = SARIMAXmodel.fit()

y_pred = SARIMAXmodel.get_forecast(len(test.index))
y_pred_df = y_pred.conf_int(alpha = 0.05)
y_pred_df["Predictions"] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
y_pred_df.index = test.index
y_pred_out = y_pred_df["Predictions"]
plt.plot(y_pred_out, color='Blue', label = 'SARIMA Predictions')
plt.legend()